In [2]:
import pandas as pd

data = pd.read_csv("./database", parse_dates=["Date"]).iloc[:, 1:]
data = data.loc[data["Date"] >= "2021-06-01"]

In [203]:
variance = {}



for comp in data["Symbol"].unique():
    curr_comp = data.loc[data["Symbol"] == comp]
    change_percent = (curr_comp["High"] - curr_comp["Low"])/curr_comp["Close"]
    variance[comp] = change_percent.mean()*100
    
avg_change = {k: v for k, v in sorted(variance.items(), key=lambda item: item[1])}
avg_change = pd.DataFrame.from_dict(avg_change, orient='index', columns = ["Average Fluctuation"])
avg_change.tail(10)

,Average Fluctuation
ICFC,7.769967
UNHPL,7.831952
JOSHI,7.840922
GHL,7.904054
MHNL,7.976492
KPCL,7.976708
UPCL,8.045708
PROFL,8.061567
SAHAS,8.064908
MKJC,8.153206


In [217]:
variance = {}

data = data.loc[data["Date"] >= "2021-06-01"]


for comp in data["Symbol"].unique():
    curr_comp = data.loc[data["Symbol"] == comp]
    change_percent = (curr_comp["High"] - curr_comp["Low"])/curr_comp["Close"]
    variance[comp] = change_percent.mean()*100
    
avg_change = {k: v for k, v in sorted(variance.items(), key=lambda item: item[1])}
avg_change = pd.DataFrame.from_dict(avg_change, orient='index', columns = ["Average Fluctuation"])
avg_change.tail(10)

,Average Fluctuation
ICFC,7.769967
UNHPL,7.831952
JOSHI,7.840922
GHL,7.904054
MHNL,7.976492
KPCL,7.976708
UPCL,8.045708
PROFL,8.061567
SAHAS,8.064908
MKJC,8.153206


In [64]:
for comp in data["Symbol"].unique():
    curr_comp = data.loc[data["Symbol"] == comp]
    change_percent = (curr_comp["High"] - curr_comp[["Open", "Close"]].max(axis=1))/curr_comp[["Open", "Close"]].max(axis = 1) \
                        - (curr_comp["Low"] - curr_comp[["Open", "Close"]].min(axis=1))/curr_comp[["Open", "Close"]].min(axis = 1)
    variance[comp] = change_percent.mean()*100
    
avg_change = {k: v for k, v in sorted(variance.items(), key=lambda item: item[1])}
avg_change = pd.DataFrame.from_dict(avg_change, orient='index', columns = ["Trading Window Fluctuation"])
avg_change.tail(10)

,Trading Window Fluctuation
KRBL,3.740515
NABBC,3.755623
RURU,3.811858
UPCL,3.816388
UMRH,4.102580
PROFL,4.154732
GRDBL,4.175816
MHNL,4.209870
GLH,4.422497
KPCL,4.438537


In [169]:
data = data.loc[data["Date"] >= "2021-07-01"]

# data = data.iloc[:, 1:]
bfc_data = data.loc[data["Symbol"] == "AKPL"]
# nepse_data = data.loc[data["Symbol"] == "NEPSE"]
# (bfc_data["Low"] - bfc_data["Close"].shift(1)).std(), (bfc_data["Low"] - bfc_data["Close"].shift(1)).mean()

In [289]:
from finta import TA
from tabulate import tabulate

stock = "NABIL"
bfc_data = data.loc[data["Symbol"] == stock].set_index("Date").sort_index()
usd = round(((bfc_data["High"] - bfc_data["Close"].shift(1))/bfc_data["Close"].shift(1)).std()*100, 2)
lsd = round((-(bfc_data["Low"] - bfc_data["Close"].shift(1))/bfc_data["Close"].shift(1)).std()*100, 2)
# print( "U.S.D.\t:", usd)
# print( "L.S.D.\t:", lsd)
# print("U. Mean\t:\t", round(((bfc_data["High"] - bfc_data["Close"].shift(1))/bfc_data["Close"].shift(1)).mean()*100, 2))
# print("L. Mean\t:\t", round((-(bfc_data["Low"] - bfc_data["Close"].shift(1))/bfc_data["Close"].shift(1)).mean()*100, 2))

ta_piv = TA.PIVOT_FIB(bfc_data).tail(1)

pivots = {
    "R3" : ta_piv["r3"].values[0],
    "R2" : ta_piv["r2"].values[0],
    "R1" : ta_piv["r1"].values[0],
    "Pivot" : ta_piv["pivot"].values[0],
    "S1" : ta_piv["s1"].values[0],
    "S2" : ta_piv["s2"].values[0],
    "S3" : ta_piv["s3"].values[0]
}

print(tabulate([(k, round(v, 1)) for k,v in pivots.items()], headers = [stock, "Price"],  tablefmt="pretty"))

print("LTP \t\t: ", round(bfc_data["Close"].iloc[-1]))
print("Buy Price\t: ", round(bfc_data["Close"].iloc[-1]*(1-lsd/100), 2))
print("Sell Price\t: ", round(bfc_data["Close"].iloc[-1]*(1+usd/100), 2))


+-------+--------+
| NABIL | Price  |
+-------+--------+
|  R3   | 1580.0 |
|  R2   | 1571.6 |
|  R1   | 1566.4 |
| Pivot | 1558.0 |
|  S1   | 1549.6 |
|  S2   | 1544.4 |
|  S3   | 1536.0 |
+-------+--------+
LTP 		:  1547
Buy Price	:  1528.13
Sell Price	:  1569.12


In [197]:

stock_data = data.loc[data["Symbol"] == "AHPC"]
nepse_data = data.loc[data["Symbol"] == "NEPSE"]
stock_data = stock_data.set_index("Date")
nepse_data = nepse_data.set_index("Date")
corr = pd.concat([nepse_data["Close"], stock_data["Close"]], axis = 1)
corr.columns = ["NEPSE", "stock"]
corr["NEPSE"].corr(corr["stock"])

0.6408173621307836

In [198]:
corr_list = {}
data.dropna()
stock = "MFIL"
stock_data = data.loc[data["Symbol"] == stock]

for sym in data["Symbol"].unique():
    if sym != stock:
        oth_data = data.loc[data["Symbol"] == sym]
        oth_data = oth_data.set_index("Date")
        corr_df = pd.concat([stock_data["Close"], oth_data["Close"]], axis = 1)        
        corr_df.columns = [sym, stock]
        corr_list[sym] =  corr_df[sym].corr(corr_df[stock])
    
corr_table = pd.DataFrame.from_dict(corr_list, orient='index', columns = ["Correlation"]).sort_values("Correlation").dropna()

In [1]:
# data.loc[data["Symbol"]=="SHINE"].set_index("Date").Close.plot(grid = True) 
# data.loc[data["Symbol"]=="BFC"].set_index("Date").Close.plot(grid=True) 
corr_table.tail(20)

NameError: name 'corr_table' is not defined

In [16]:
stock = "API"

stock_data = data.loc[data["Symbol"] == stock].sort_values("Date")
stock_data.loc[stock_data["Date"] >= "2021-10-28"].tail(5)

,Symbol,Date,Open,High,Low,Close,Percent Change,Volume
66129,API,2021-11-16,507.0,525.0,497.0,521.0,2.76,455303.0
249154,API,2021-11-16,507.0,525.0,497.0,521.0,2.76,455303.0
249153,API,2021-11-17,531.4,531.8,509.0,510.0,-2.11,388095.0
66130,API,2021-11-17,531.4,531.8,509.0,510.0,-2.11,388095.0
66131,API,2021-11-18,501.0,516.0,500.0,503.4,-1.29,217470.0
